In [1]:
# SG by category model 
using Pkg, JuMP, Ipopt, DataFrames, CSV, Random, Distributions

In [9]:
#pweight_df = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/realv2_plyr_weights.csv") # nodsr weights
#cweight_df = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/realv2_course_weights.csv")


pweight_df = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/realv2_total_scaled_round_sg_plyr_weights.csv") # dsr weights
cweight_df = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/realv2_total_scaled_round_sg_course_weights.csv")

real_data = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/scaled_round_sg_2018.csv") # scaled data

real_data[!, :SG_total] = real_data[!, :SG_appr] + real_data[!, :SG_short]  +real_data[!, :SG_drive] + real_data[!, :SG_putt] 
n_real = length(real_data[!,:player])
num_plyrs = length(unique(real_data[!,:player]))
num_courses = length(unique(real_data[!,:course_name]))

37

# NO DSR 

In [6]:
mod_real = Model(with_optimizer(Ipopt.Optimizer))

@variable(mod_real, p_sg_total[1:num_plyrs])

@variable(mod_real, c_sg_total[1:num_courses])


p_sg_total_ = [p_sg_total[i+1] for i in real_data[!,:p_idx]]

c_sg_total_ = [c_sg_total[i+1] for i in real_data[!,:c_idx]]

hsg_wmean_total = @expression(mod_real, sum(c_sg_total[i]*(cweight_df[!,:weight][i]) for i in 1:num_courses))

hsg_total_avg_constr = @constraint(mod_real,  hsg_wmean_total == 0.0) 

psg_wmean_total = @expression(mod_real, sum(p_sg_total[i]*(pweight_df[!,:weight][i]) for i in 1:num_plyrs))

psg_total_avg_constr = @constraint(mod_real,  psg_wmean_total == 0.0) 

sg_total_constr = @constraint(mod_real, [i=1:num_courses], -5 <= c_sg_total[i] <= 5) 

psg_total_constr = @constraint(mod_real, [i=1:num_plyrs], -5 <= p_sg_total[i] <= 5) 


@NLobjective(mod_real, Min, sum( (p_sg_total_[i]*p_sg_total_[i] - 2*p_sg_total_[i]*c_sg_total_[i] + c_sg_total_[i]*c_sg_total_[i] - 2*p_sg_total_[i]*real_data[!,:SG_total][i] + 2*c_sg_total_[i]*real_data[!,:SG_total][i] + real_data[!,:SG_total][i]*real_data[!,:SG_total][i])  for i in 1:n_real)/(n_real))

status_real = optimize!(mod_real)

c_idx_df = DataFrame(idx = unique(real_data[!,:c_idx]), course_name = unique(real_data[!,:course_name]))
p_idx_df = DataFrame(idx = unique(real_data[!,:p_idx]), player = unique(real_data[!,:player]))

p_t_label = [value.(p_sg_total)[i+1] for i in p_idx_df[!,:idx]]

c_t_label = [value.(c_sg_total)[i+1] for i in c_idx_df[!,:idx]]

p_rounds = [pweight_df[!,:weight][i+1]*n_real for i in p_idx_df[!,:idx]]
c_rounds = [cweight_df[!,:weight][i+1]*n_real for i in c_idx_df[!,:idx]]
nodsr_obj_val = DataFrame(obj = [sqrt(objective_value(mod_real))])

nodsr_c_sg_final_df = DataFrame(idx = c_idx_df[!,:idx], course_name = c_idx_df[!,:course_name], sg_total = c_t_label,num_rounds = c_rounds)

nodsr_p_final_df = DataFrame(idx = p_idx_df[!,:idx], player = p_idx_df[!,:player], sg_total = p_t_label, num_rounds = p_rounds)


CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_total_plyr_nodsr.csv", nodsr_p_final_df)
CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_total_course_nodsr.csv", nodsr_c_sg_final_df)
CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_total_obj_nodsr.csv", nodsr_obj_val)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      541
Number of nonzeros in inequality constraint Jacobian.:     1082
Number of nonzeros in Lagrangian Hessian.............:     5360

Total number of variables............................:      541
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:     1082
        inequality constraints with only lower bounds:      541
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      541

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

"C:/Users/Luke/OneDrive/Research/sg_data/realv2_total_obj_nodsr.csv"

#  DSR

In [10]:
## have to run preprocessing to get weights from no dsr first
### WORKING DSR MODEL: 08-24-2020

mod_real = Model(with_optimizer(Ipopt.Optimizer))

@variable(mod_real, p_sg_total[1:num_plyrs])
@variable(mod_real, c_sg_total[1:num_courses])
@variable(mod_real, c_dsr_total[1:num_courses])


p_sg_total_ = [p_sg_total[i+1] for i in real_data[!,:p_idx]]
c_sg_total_ = [c_sg_total[i+1] for i in real_data[!,:c_idx]]
c_dsr_total_ = [c_dsr_total[i+1] for i in real_data[!,:c_idx]]



hsg_wmean_total = @expression(mod_real, sum(c_sg_total[i]*(filter(row -> (row.c_idx == i-1) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
hsg_total_avg_constr = @constraint(mod_real,  hsg_wmean_total == 0.0) 


psg_wmean_total = @expression(mod_real, sum(p_sg_total[i]*(filter(row -> (row.p_idx == i-1) , pweight_df)[!,:weight][1]) for i in 1:num_plyrs))
psg_total_avg_constr = @constraint(mod_real,  psg_wmean_total == 0.0) 

real_pwvar_total = @NLexpression(mod_real, (num_plyrs/(num_plyrs-1))*sum(((filter(row -> (row.p_idx == i-1) , pweight_df)[!,:weight][1])*(abs(p_sg_total[i]))*(abs(p_sg_total[i]))) for i in 1:num_plyrs)) # weighted variance
real_ptotal_var_constr = @NLconstraint(mod_real,  real_pwvar_total == 1.0)

dsr_total_constr = @constraint(mod_real, [i=1:num_courses], 0 <= c_dsr_total[i] <= 4) 

sg_total_apx_constr = @constraint(mod_real, [i=1:num_courses], filter(row -> (row.c_idx == i-1) , cweight_df)[!,:sg_total_mean][1] -3 <= c_sg_total[i] <= filter(row -> (row.c_idx == i-1) , cweight_df)[!,:sg_total_mean][1] + 3) 
psg_total_apx_constr = @constraint(mod_real, [i=1:num_plyrs], filter(row -> (row.p_idx == i-1) , pweight_df)[!,:sg_total_mean][1]-3 <= p_sg_total[i] <= filter(row -> (row.p_idx == i-1) , pweight_df)[!,:sg_total_mean][1]+3)


# p*d - c
@NLobjective(mod_real, Min, sum( (p_sg_total_[i]*p_sg_total_[i]*c_dsr_total_[i]*c_dsr_total_[i] - 2*p_sg_total_[i]*c_dsr_total_[i]*c_sg_total_[i] + c_sg_total_[i]*c_sg_total_[i]- 2*p_sg_total_[i]*c_dsr_total_[i]*real_data[!,:SG_total][i] + 2*c_sg_total_[i]*real_data[!,:SG_total][i] + real_data[!,:SG_total][i]*real_data[!,:SG_total][i])  for i in 1:n_real)/(n_real))

status_real = optimize!(mod_real)

c_idx_df = DataFrame(idx = unique(real_data[!,:c_idx]), course_name = unique(real_data[!,:course_name]))
p_idx_df = DataFrame(idx = unique(real_data[!,:p_idx]), player = unique(real_data[!,:player]))

p_t_label = [value.(p_sg_total)[i+1] for i in p_idx_df[!,:idx]]
c_t_label = [value.(c_sg_total)[i+1] for i in c_idx_df[!,:idx]]
cd_t_label = [value.(c_dsr_total)[i+1] for i in c_idx_df[!,:idx]]


rounds = [filter(row -> (row.p_idx == i) , pweight_df)[!,:weight][1]*n_real for i in p_idx_df[!,:idx]]
obj_val =  objective_value(mod_real)

c_sg_final_df = DataFrame(c_idx = c_idx_df[!,:idx], course_name = c_idx_df[!,:course_name], sg_total = c_t_label)
c_dsr_final_df = DataFrame(c_idx = c_idx_df[!,:idx], course_name = c_idx_df[!,:course_name], dsr_total = cd_t_label)

p_final_df = DataFrame(p_idx = p_idx_df[!,:idx], player = p_idx_df[!,:player], sg_total = p_t_label, num_rounds = rounds)

obj = DataFrame(obj = obj_val)


CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_total_2018_plyr_.csv", p_final_df)
CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_total_2018_csg_sg_.csv", c_sg_final_df)
CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_total_2018_c_sg_.csv", c_dsr_final_df)
CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_total_2018_obj_.csv", obj)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     1045
Number of nonzeros in inequality constraint Jacobian.:     1156
Number of nonzeros in Lagrangian Hessian.............:    10757

Total number of variables............................:      578
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        3
Total number of inequality constraints...............:     1156
        inequality constraints with only lower bounds:      578
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      578

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

"C:/Users/Luke/OneDrive/Research/sg_data/realv2_total_2018_obj_.csv"